In [ ]:
import os
# 기본 경로
base_dir = './images/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# 훈련에 사용되는 남자 여자 이미지 경로
train_female_dir = os.path.join(train_dir, 'female')
train_male_dir = os.path.join(train_dir, 'male')
print(train_female_dir)
print(train_male_dir)

# 테스트에 사용되는 남자 여자 이미지 경로
validation_female_dir = os.path.join(validation_dir, 'female')
validation_male_dir = os.path.join(validation_dir, 'male')
print(validation_female_dir)
print(validation_male_dir)

./images/train/female
./images/train/male
./images/validation/female
./images/validation/male


In [ ]:
train_female_fnames = os.listdir( train_female_dir )
train_male_fnames = os.listdir( train_male_dir )

print(train_female_fnames[:5])
print(train_male_fnames[:5])

['1237.png', '2136.png', '2187.png', '1473.png', '1346.png']
['3694.png', '2360.png', '2629.png', '0650.png', '1956.png']


In [ ]:
print('Total training female images :', len(os.listdir(train_female_dir)))
print('Total training male images :', len(os.listdir(train_male_dir)))

print('Total validation female images :', len(os.listdir(validation_female_dir)))
print('Total validation male images :', len(os.listdir(validation_male_dir)))

Total training cat images : 1000
Total training dog images : 1000
Total validation cat images : 500
Total validation dog images : 500


In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

nrows, ncols = 4, 4
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols*3, nrows*3)

pic_index+=8

next_female_pix = [os.path.join(train_female_dir, fname)
                for fname in train_female_fnames[ pic_index-8:pic_index]]

next_male_pix = [os.path.join(train_male_dir, fname)
                for fname in train_male_fnames[ pic_index-8:pic_index]]

for i, img_path in enumerate(next_female_pix+next_male_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
import tensorflow as tf


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
            loss='binary_crossentropy',
            metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                  batch_size=20,
                                                  class_mode='binary',
                                                  target_size=(150, 150))
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                       batch_size=20,
                                                       class_mode  = 'binary',
                                                       target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=100,
                    epochs=100,
                    validation_steps=50,
                    verbose=2)

In [ ]:
!nvidia-smi

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'go', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():

  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))

  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)

  print(classes[0])

  if classes[0]>0:
    print(fn + " is a male")

  else:
    print(fn + " is a female")

Saving 2.jpg to 2.jpg
[1.]
2.jpg is a male


In [ ]:
from keras.models import load_model
model.save('mnist_mlp_model.h5')

In [ ]:
from keras.models import load_model
model = load_model('mnist_mlp_model.h5')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))